In [ ]:
# 数据清洗

In [ ]:
import glob
import pandas as pd
from pathlib import Path
import re

In [ ]:
from PIL import Image

In [ ]:
model_label_map = {
    0: 10,  # 鞋子
    1: 8,  # 包包
    2: 4,  # 上装
    3: 5,  # 裤子
    4: 6,  # 裙子
    5: 7,  # 连体装
}

In [ ]:
model_label_map = {
    0: 'shoe',
    1: 'bag',
    2: 'upperbody',
    3: 'lowerbody',  # 裤子
    4: 'skirt',  # 裙子
    5: 'wholebody',  # 连体装
}

In [ ]:
from tqdm import tqdm
def df_map(df):
    names = df.columns
    k_list = df[df.columns[0]].values.tolist()
    v_list = df[df.columns[1]].values.tolist()
    d = {}
    for k, v in tqdm(zip(k_list, v_list)):
        d[k] = v
    return d

In [ ]:
#  dataframe multi-thread
from pandarallel import pandarallel
# Initialization
pandarallel.initialize()
# df.parallel_apply(func, progress_bar=True)

### 1.LOAD DATA

In [ ]:
bboxes_all_df = pd.read_csv('/home/chenyi/workspace/myPyModule99/gallery_manage/gallery_clr_pipeline/data_temp/bboxes_clred_temp.csv')
# bboxes_all_df.to_csv('./data_temp/bboxes_all_df.csv', index=False)
len(bboxes_all_df)

In [ ]:
len(bboxes_all_df)

In [ ]:
# %%time
# bboxes_all_df['stem_crop'] = bboxes_all_df['crop_path'].parallel_apply(lambda x: Path(x).stem)

In [ ]:
# clred_df = pd.read_csv('./data_temp/bboxes_clred_upper.csv')
# clred_df.to_csv('./data_temp/bboxes_clred_final.csv', index=False)
# len(clred_df)

In [ ]:
# bboxes_all_df.value_counts('label_pred').sort_index()

#### det drawed image

In [ ]:
# imgs_det_df.to_csv('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/gall7kv6/info/imgs_gall_df.csv', index=False)
imgs_gall_df = pd.read_csv('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/gall7kv6/info/imgs_gall_df.csv')
len(imgs_gall_df)

In [ ]:
df = imgs_gall_df.loc[:, ['stem', 'det_path']]
stem2det_path = df_map(df)
del imgs_gall_df

#### crops target path.

In [ ]:
# crops_det_df = pd.read_csv('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/gall7kv6/info/crops_det_df.csv')
# txts_gall_df = pd.read_csv('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/gall7kv6/info/txts_gall_df.csv')

#### df预处理

In [ ]:
import pickle
with open('/nas/chenyi/datasets_cls/map/gallery/stem2width_gallery2m20220728.pickle', 'rb') as f:
    stem2width = pickle.load(f)
with open('/nas/chenyi/datasets_cls/map/gallery/stem2height_gallery2m20220728.pickle', 'rb') as f:
    stem2height = pickle.load(f)

In [ ]:
bboxes_all_df['width'] = bboxes_all_df['stem'].map(stem2width)
bboxes_all_df['height'] = bboxes_all_df['stem'].map(stem2height)

In [ ]:
bboxes_all_df.columns

In [ ]:
## oss path cleared image.
gall_df = pd.read_csv('/nas/chenyi/datasets_cls/gallery_detect/gallery_csv/gall2m_hw_df20220725.csv')
len(gall_df)
gall_clred_df = gall_df[gall_df['state']>=0]
len(gall_clred_df)

In [ ]:
clred_df = bboxes_all_df[bboxes_all_df['stem'].isin(gall_clred_df['stem'])]
len(clred_df)

In [ ]:
# crop image exist.
clred_df = clred_df[clred_df['crop_exist']]
len(clred_df)

In [ ]:
# clred_df.drop(columns='crop_exist', inplace=True)

In [ ]:
clred_df['bx_w'] = clred_df['width'].values*clred_df['bx3'].values
clred_df['bx_h'] = clred_df['height'].values*clred_df['bx4'].values
clred_df['h/w'] = clred_df['bx_h'].values/clred_df['bx_w'].values

### 2.图片清洗

In [ ]:
# clred_df = bboxes_all_df
# len(clred_df)

#### 图片太小

In [ ]:
thres = 300
temp = clred_df[(clred_df['width']<thres) | (clred_df['height']<thres)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem'].isin(temp['stem'])]
len(temp)/len(bboxes_all_df), len(clred_df), len(bboxes_all_df)

In [ ]:
thres = 400
temp = clred_df[(clred_df['width']<thres) & (clred_df['height']<thres)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem'].isin(temp['stem'])]
len(temp)/len(bboxes_all_df), len(clred_df), len(bboxes_all_df)

#### 去除置信度太小的

In [ ]:
clred_df['conf_show'] = clred_df['conf'].apply(lambda x: round(x*10))

In [ ]:
clred_df.value_counts('conf_show').sort_index()

In [ ]:
thres = 0.5
clred_df =  clred_df[(clred_df['conf']>=thres)]
len(clred_df)

In [ ]:
clred_df.drop(columns='conf_show', inplace=True)

#### 去掉图片目标太多的

In [ ]:
thres = 7
temp = clred_df[clred_df['crop_id']>7]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem'].isin(temp['stem'])]
len(temp)/len(bboxes_all_df), len(clred_df), len(bboxes_all_df)

### 3.bbox整体处理

In [ ]:
# bboxes_all_df = clred_df

#### 目标太小

In [ ]:
thres = 20
clred_df = clred_df[(clred_df['bx_w']>thres) & (clred_df['bx_h']>thres)]

In [ ]:
len(bboxes_all_df), len(clred_df), len(clred_df)/len(bboxes_all_df)

#### bbox h/w

In [ ]:
thres = 10
temp = clred_df[(clred_df['h/w']>thres) | (clred_df['h/w']<1/thres)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

### 4.类别单独处理

#### 4.0 Shoe

In [ ]:
# clred_df = bboxes_all_df
cls_check = 0

##### confidence

In [ ]:
thres = 0.7
temp = clred_df[(clred_df['conf']<thres)
               & (clred_df['label_pred']==cls_check)]
len(temp),len(clred_df[clred_df['label_pred']==cls_check])

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

##### 类别bbox太小

In [ ]:
thres = 20
temp = clred_df[(clred_df['bx_h']<thres) | (clred_df['bx_w']<thres) 
             & (clred_df['label_pred']==cls_check)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

In [ ]:
# bbox相对值大小
w_thres = 0.02
h_thres = 0.025
temp = clred_df[(clred_df['bx3']<w_thres) | (clred_df['bx4']<h_thres) 
             & (clred_df['label_pred']==cls_check)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

##### 长宽比

In [ ]:
# thres_max = 7
# thres_min = 0.25
# temp = clred_df[((clred_df['h/w']>thres_max) | (clred_df['h/w']<thres_min ))
#              & (clred_df['label_pred']==cls_check)]
# len(temp)

In [ ]:
# clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
# len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

##### 鞋子的位置

In [ ]:
h_max_thres = 0.98
h_min_thres = 0.55
w_max_thres = 0.95
w_min_thres = 0.05

temp = clred_df[((clred_df['bx1']>w_max_thres) | 
                (clred_df['bx1']<w_min_thres) |
                (clred_df['bx2']>h_max_thres) | 
                (clred_df['bx2']<h_min_thres) )
             & (clred_df['label_pred']==cls_check)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

##### 只要一双鞋

In [ ]:
# 该版本暂时不管鞋子

In [ ]:
clred_df.head(1)

In [ ]:
# temp = clred_df[(clred_df['crop_id']>=2)  & 
#                 (clred_df['label_pred']==cls_check)]
# len(temp)

In [ ]:
# temp1 = clred_df[clred_df['stem'].isin(temp['stem'])]
# len(temp1)

In [ ]:
temp1.sort_values(['stem', 'conf'], inplace=True)

In [ ]:
temp.head(10)

##### csv保存

In [ ]:
clred_df.to_csv('./data_temp/bboxes_clred_shoe.csv', index=False)

#### 4.1 bag

In [ ]:
# clred_df = pd.read_csv('./data_temp/bboxes_clred_bag.csv')

In [ ]:
# bboxes_all_df = clred_df

In [ ]:
cls_check = 1

##### confidence

In [ ]:
thres = 0.68
temp = clred_df[(clred_df['conf']<thres)
               & (clred_df['label_pred']==cls_check)]
len(temp),len(clred_df[clred_df['label_pred']==cls_check])

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

##### 类别bbox太小

In [ ]:
#  bbox绝对值大小
thres = 20
temp = clred_df[(clred_df['bx_h']<thres) | (clred_df['bx_w']<thres) 
             & (clred_df['label_pred']==cls_check)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

In [ ]:
# bbox相对值大小
thres_h = 0.075
thres_w = 0.05
temp = clred_df[(clred_df['bx3']<thres_h) & (clred_df['bx4']<thres_w )
             & (clred_df['label_pred']==cls_check)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df)

##### 长宽比

In [ ]:
thres_max = 7
thres_min = 1/2.5
temp = clred_df[((clred_df['h/w']>thres_max) | (clred_df['h/w']<thres_min ))
             & (clred_df['label_pred']==cls_check)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df)

##### 位置

In [ ]:
h_max_thres = 0.925
h_min_thres = 0.25
w_max_thres = 0.95
w_min_thres = 0.05

temp = clred_df[((clred_df['bx1']>w_max_thres) | 
                (clred_df['bx1']<w_min_thres) |
                (clred_df['bx2']>h_max_thres) | 
                (clred_df['bx2']<h_min_thres) )
             & (clred_df['label_pred']==cls_check)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df)

##### 只能有一个bag

In [ ]:
clred_bag_df = clred_df[clred_df['label_pred']==cls_check]

In [ ]:
# clred_bag_df.sort_values(['len', 'stem'], ascending=False, inplace=True)

In [ ]:
bag_gp = dict(list(clred_bag_df.groupby('stem')))

In [ ]:
%%time
stem_crop_list = []
for stem, v in tqdm(bag_gp.items()):
    if len(v)>=2:
        v.sort_values('conf', ascending=False, inplace=True)
        item = v.drop_duplicates('stem', keep='first')
    else:
        item = v
    stem_crop_list.append(item['stem_crop'].values[0])
    # break

In [ ]:
temp = clred_bag_df[~clred_bag_df['stem_crop'].isin(stem_crop_list)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df)

##### bag csv save

In [ ]:
clred_df.to_csv('./data_temp/bboxes_clred_bag.csv', index=False)

#### 4.2 upperbody

In [ ]:
# clred_df = pd.read_csv('./data_temp/bboxes_clred_bag.csv')
clred_df.to_csv('./data_temp/bboxes_clred_temp1.csv', index=False)

In [ ]:
cls_check = 2

##### confidence

In [ ]:
thres = 0.75
temp = clred_df[(clred_df['conf']<thres)
               & (clred_df['label_pred']==cls_check)]
len(temp),len(clred_df[clred_df['label_pred']==cls_check])

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

##### bbox大小

In [ ]:
#  bbox绝对值大小 or
thres_h = 40
thres_w = 40
temp = upperbody_df[(upperbody_df['bx_h']<thres_h) | (upperbody_df['bx_w']<thres_w)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

In [ ]:
#  bbox绝对值大小 and
thres_w = 50
thres_h = 65
temp = upperbody_df[(upperbody_df['bx_h']<thres_h) & (upperbody_df['bx_w']<thres_w)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

In [ ]:
# bbox相对值大小
w_thres = 0.1
h_thres = 0.07
temp = upperbody_df[(upperbody_df['bx3']<w_thres) | (upperbody_df['bx4']<h_thres)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

In [ ]:
# bbox相对值大小
w_thres = 0.1
h_thres = 0.07
temp = upperbody_df[(upperbody_df['bx3']<w_thres) & (upperbody_df['bx4']<h_thres)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

##### 长宽比

In [ ]:
thres_max = 3
thres_min = 0.25
temp = clred_df[((clred_df['h/w']>thres_max) | (clred_df['h/w']<thres_min )) 
                # & (clred_df['bx_h']>500) 
                # & (clred_df['bx_w']<50 )
             & (clred_df['label_pred']==cls_check)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

##### 位置

In [ ]:
h_max_thres = 0.85
h_min_thres = 0.175
w_max_thres = 0.95
w_min_thres = 0.05

temp = clred_df[((clred_df['bx1']>w_max_thres) | 
                (clred_df['bx1']<w_min_thres) |
                (clred_df['bx2']>h_max_thres) | 
                (clred_df['bx2']<h_min_thres) )
             & (clred_df['label_pred']==cls_check)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

##### 上装数目太多

In [ ]:
# 可以考虑只保留中心的衣服

In [ ]:
upper_gp = dict(list(upperbody_df.groupby('stem')))

In [ ]:
%%time
thres = 3
much_item_list = []
len_list = []
for stem, v in tqdm(upper_gp.items()):
    if len(v)>=thres:
        much_item_list.append(stem)
        len_list.append(len(v))
        # break

In [ ]:
len(much_item_list)

In [ ]:
temp = upperbody_df[upperbody_df['stem'].isin(much_item_list)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df)

##### CSV SAVE

In [ ]:
clred_df.to_csv('./data_temp/bboxes_clred_upper.csv', index=False)

#### 4.3 lowerbody

In [ ]:
cls_check = 3

In [ ]:
# bboxes_all_df = clred_df

##### confidence

In [ ]:
thres = 0.75
temp = clred_df[(clred_df['conf']<thres)
               & (clred_df['label_pred']==cls_check)]
len(temp),len(clred_df[clred_df['label_pred']==cls_check])

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

##### bbox大小

In [ ]:
#  bbox绝对值大小 or
thres_h = 30
thres_w = 40
temp = clred_df[((clred_df['bx_h']<thres_h) | (clred_df['bx_w']<thres_w))
               & (clred_df['label_pred']==cls_check)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

In [ ]:
#  bbox绝对值大小 and
thres_w = 45
thres_h = 55
temp = clred_df[((clred_df['bx_h']<thres_h) & (clred_df['bx_w']<thres_w))
               & (clred_df['label_pred']==cls_check)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

In [ ]:
# bbox相对值大小
w_thres = 0.1
h_thres = 0.05
temp = clred_df[((clred_df['bx3']<w_thres) | (clred_df['bx4']<h_thres))
                   & (clred_df['label_pred']==cls_check)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

##### 位置

In [ ]:
h_min_thres = 0.3
h_max_thres = 0.9
w_max_thres = 0.95
w_min_thres = 0.05

temp = clred_df[((clred_df['bx1']>w_max_thres) | 
                (clred_df['bx1']<w_min_thres) |
                (clred_df['bx2']>h_max_thres) | 
                (clred_df['bx2']<h_min_thres) )
             & (clred_df['label_pred']==cls_check)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

##### 长宽比

In [ ]:
thres_max = 4.5
thres_min = 0.35
temp = clred_df[((clred_df['h/w']>thres_max) | (clred_df['h/w']<thres_min )) 
             & (clred_df['label_pred']==cls_check)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

##### 数目太多

In [ ]:
lower_df = clred_df[clred_df['label_pred']==cls_check]

In [ ]:
lower_gp = dict(list(lower_df.groupby('stem')))

In [ ]:
%%time
thres = 2
much_item_list = []
len_list = []
for stem, v in tqdm(lower_gp.items()):
    if len(v)>=thres:
        much_item_list.append(stem)
        len_list.append(len(v))
        # break

In [ ]:
len(much_item_list)

In [ ]:
temp = lower_df[lower_df['stem'].isin(much_item_list)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df)

In [ ]:
# clred_df.to_csv('./data_temp/bboxes_clred_lower.csv', index=False)

#### 4.4 SKIRT

In [ ]:
# clred_df = pd.read_csv('./data_temp/bboxes_clred_lower.csv')
# bboxes_all_df = clred_df

In [ ]:
cls_check = 4

##### confidence

In [ ]:
thres = 0.8
temp = clred_df[(clred_df['conf']<thres)
               & (clred_df['label_pred']==cls_check)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

##### bbox大小

In [ ]:
#  bbox绝对值大小 or
thres_h = 30
thres_w = 40
temp = clred_df[((clred_df['bx_h']<thres_h) | (clred_df['bx_w']<thres_w))
               & (clred_df['label_pred']==cls_check)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

In [ ]:
#  bbox绝对值大小 and
thres_w = 45
thres_h = 55
temp = clred_df[((clred_df['bx_h']<thres_h) & (clred_df['bx_w']<thres_w))
               & (clred_df['label_pred']==cls_check)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

In [ ]:
# bbox相对值大小
w_thres = 0.1
h_thres = 0.05
temp = clred_df[((clred_df['bx3']<w_thres) | (clred_df['bx4']<h_thres))
                   & (clred_df['label_pred']==cls_check)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

##### 长宽比

In [ ]:
thres_max = 4.5
thres_min = 0.35
temp = clred_df[((clred_df['h/w']>thres_max) | (clred_df['h/w']<thres_min )) 
             & (clred_df['label_pred']==cls_check)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

##### 位置

In [ ]:
h_min_thres = 0.4
h_max_thres = 0.9
w_max_thres = 0.95
w_min_thres = 0.05

temp = clred_df[((clred_df['bx1']>w_max_thres) | 
                (clred_df['bx1']<w_min_thres) |
                (clred_df['bx2']>h_max_thres) | 
                (clred_df['bx2']<h_min_thres) )
             & (clred_df['label_pred']==cls_check)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

##### 数目

In [ ]:
lower_df = clred_df[clred_df['label_pred']==cls_check]

In [ ]:
lower_gp = dict(list(lower_df.groupby('stem')))

In [ ]:
%%time
thres = 2
much_item_list = []
len_list = []
for stem, v in tqdm(lower_gp.items()):
    if len(v)>=thres:
        much_item_list.append(stem)
        len_list.append(len(v))
        # break

In [ ]:
len(much_item_list)

In [ ]:
temp = lower_df[lower_df['stem'].isin(much_item_list)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df)

#### 4.5 wholebody

In [ ]:
cls_check = 5

In [ ]:
thres = 0.7
temp = clred_df[(clred_df['conf']<thres)
               & (clred_df['label_pred']==cls_check)]
len(temp), len(clred_df[clred_df['label_pred']==cls_check])

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

##### bbox大小

In [ ]:
#  bbox绝对值大小 or
thres_h = 150
thres_w = 60
temp = clred_df[((clred_df['bx_h']<thres_h) | (clred_df['bx_w']<thres_w))
               & (clred_df['label_pred']==cls_check)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

In [ ]:
#  bbox绝对值大小 and
thres_w = 75
thres_h = 200
temp = clred_df[((clred_df['bx_h']<thres_h) & (clred_df['bx_w']<thres_w))
               & (clred_df['label_pred']==cls_check)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

In [ ]:
# bbox相对值大小
w_thres = 0.1
h_thres = 0.2
temp = clred_df[((clred_df['bx3']<w_thres) | (clred_df['bx4']<h_thres))
                   & (clred_df['label_pred']==cls_check)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

##### 长宽比

In [ ]:
thres_max = 4
thres_min = 1
temp = clred_df[((clred_df['h/w']>thres_max) | (clred_df['h/w']<thres_min )) 
             & (clred_df['label_pred']==cls_check)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

##### 位置

In [ ]:
h_min_thres = 0.3
h_max_thres = 0.85
w_max_thres = 0.95
w_min_thres = 0.05

temp = clred_df[((clred_df['bx1']>w_max_thres) | 
                (clred_df['bx1']<w_min_thres) |
                (clred_df['bx2']>h_max_thres) | 
                (clred_df['bx2']<h_min_thres) )
             & (clred_df['label_pred']==cls_check)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df),  len(bboxes_all_df)

##### 数目

In [ ]:
lower_df = clred_df[clred_df['label_pred']==cls_check]

In [ ]:
lower_gp = dict(list(lower_df.groupby('stem')))

In [ ]:
%%time
thres = 2
much_item_list = []
len_list = []
for stem, v in tqdm(lower_gp.items()):
    if len(v)>=thres:
        much_item_list.append(stem)
        len_list.append(len(v))
        # break

In [ ]:
len(much_item_list)

In [ ]:
temp = lower_df[lower_df['stem'].isin(much_item_list)]
len(temp)

In [ ]:
clred_df = clred_df[~clred_df['stem_crop'].isin(temp['stem_crop'])]
len(temp)/len(bboxes_all_df), len(clred_df)/len(bboxes_all_df)

In [ ]:
clred_df.to_csv('./data_temp/bboxes_clred_wholebody.csv', index=False)

In [ ]:
len(clred_df)

### 5.html display

In [ ]:
# stem = '1be17c97-4671-4914-a047-94b021788a79-762233'
# bboxes_all_df[bboxes_all_df['stem']==stem]
# !cat /nas/chenyi/datasets_cls/gallery_detect/gallery_oss/gall7kv6/labels/1b/e1/1be17c97-4671-4914-a047-94b021788a79-762233.txt

##### 预处理

In [ ]:
import sys
sys.path.append('../../')
from utils.display.html import *

In [ ]:
clred_df = bboxes_all_df

In [ ]:
len(clred_df)

In [ ]:
clred_df.sort_values('conf', ascending=True, inplace=True)

In [ ]:
cls_check = 5

In [ ]:
thres = 0.65
temp = clred_df[(clred_df['conf']<thres)
               & (clred_df['label_pred']==cls_check)]
len(temp), len(clred_df[clred_df['label_pred']==cls_check])

In [ ]:
dis_df = temp.sort_values('conf', ascending=False)[0:50]
# dis_df = clred_df[0:20]

In [ ]:
dis_df = dis_df.drop_duplicates('stem')

In [ ]:
!du -h ./data_temp/*

#### Display

In [ ]:
dis_df['img_path'] = dis_df['stem'].map(stem2det_path)

In [ ]:
dis_df['image'] = dis_df['img_path'].apply(lambda x: get_thumbnail(x, size=640))
dis_df['crop_img'] = dis_df['crop_path'].apply(lambda x: get_thumbnail(x, size=200))

In [ ]:
HTML(dis_df[['label_pred', 'crop_img', 'image', 'conf', 'stem']].to_html(formatters={'crop_img': image_formatter,
                                                                                             'image': image_formatter}, escape=False))

##### Data copy

In [ ]:
copy_df = temp

In [ ]:
copy_df.head(1)

In [ ]:
import shutil

root_source = Path('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/gall7kv6/images/')
root_target = Path('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/gall7kv6/clear/image_conf/wholebody/63_65/')
for stem in tqdm(copy_df['stem'].values):
    sp = root_source / stem[:2] / stem[2:4] / stem
    tp = root_target / stem[:2] / stem
    sp = sp.with_suffix('.jpg')
    tp = tp.with_suffix('.jpg')
    if tp.is_file() or not sp.is_file():
        continue
    if not tp.parent.is_dir():
        tp.parent.mkdir(parents=True, exist_ok=True)
    shutil.copyfile(sp, tp)
    # break